In [1]:
import os 
import sys
sys.path.append(os.getenv('BONNER_ROOT_PATH'))
import xarray as xr
import torch
import torchvision
import warnings
warnings.filterwarnings('ignore')

from config import ACTIVATIONS_PATH
from image_tools.processing import *
from model_evaluation.predicting_brain_data.regression.scorer import scorer
from model_evaluation.utils import get_activations_iden, get_scores_iden
from model_features.activation_extractor import Activations

# define local variables
DATASET = 'naturalscenes'
REGIONS = ['general']
MAX_POOL = True
MODE = 'ridgecv'
HOOK = None
ALPHA_RANGE = [10**i for i in range(1,10)]

In [6]:
import xarray as xr
xr.open_dataset('/data/atlas/.cache/activations/expansion_model_test_1_3_layers_10000_features_naturalscenes', 
                engine='netcdf4').stimulus_id.values

array(['image00002', 'image00003', 'image00007', ..., 'image72996',
       'image72998', 'image72999'], dtype=object)

In [2]:
from model_features.models.expansion_3_layers import Expansion

model_info = {
                'iden':'expansion_model',
                'model':Expansion(filters_3=10000).Build(),
                'layers': ['last'], 
                'num_layers':3,
                'num_features':10000,
} 

In [3]:
for region in REGIONS:
        
    activations_identifier = get_activations_iden(model_info, DATASET, MODE)
    
    activations = Activations(model=model_info['model'],
                            layer_names=model_info['layers'],
                            dataset=DATASET,
                            mode = MODE,
                            hook = HOOK,
                            batch_size = 100
                        )           

    activations.get_array(ACTIVATIONS_PATH,activations_identifier)   

    scores_identifier = get_scores_iden(model_info, activations_identifier, region, DATASET, MODE)
    
    scorer(model_name=model_info['iden'],
           activations_identifier=activations_identifier,
           scores_identifier=scores_identifier,
           dataset=DATASET,
           mode=MODE,
           region=region,
           alpha_values=ALPHA_RANGE)

extracting activations


  0%|                                 | 1/730 [00:04<58:20,  4.80s/it]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.02 GiB (GPU 0; 7.79 GiB total capacity; 4.05 GiB already allocated; 1.92 GiB free; 4.98 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF